In [915]:
import pandas as pd
import numpy as np
import os

In [916]:
#pip install pandas

In [917]:
# Helper Methods

def generate_column_info(df):
    cls = []
    nullCount = []
    nonNullCount = []
    nullsPct = []
    uniqCount = []
    dataType = []
    for i,col in enumerate(df.columns):
        cls.append(col)
        nullCount.append(df[col].isnull().sum())
        nonNullCount.append(len(df)-df[col].isnull().sum())
        nullsPct.append((df[col].isnull().sum())*(100)/len(df))
        uniqCount.append(df[col].nunique())
        dataType.append(df[col].dtype)
        
    column_info = pd.DataFrame(
        {'ColumnName': cls,
         'NullCount': nullCount,
         'NonNullCount': nonNullCount,
         'NullPercent': nullsPct,
         'UniqueValueCount': uniqCount,
         'DataType':dataType
        })
    return(column_info)

## Data

In [918]:
abbr_dict = {
    'flight_Liquefied Natural Gas ImpExp Equipment.xls':'LNG_IMEX',
    'flight_Liquefied Natural Gas Storage.xls':'LNG_STOR',
    'flight_Natural Gas Local Distribution Companies.xls':'NG_LDC',
    'flight_Natural Gas Processing.xls':'NG_PROC',
    'flight_Natural Gas TransmissionCompression.xls':'NG_COMP',
    'flight_Offshore Petroleum & Natural Gas Production.xls':'PROD_OFF',
    'flight_Onshore Natural Gas Transmission Pipelines.xls':'NG_PIPE',
    'flight_Onshore Petroleum & Natural Gas Gathering & Boosting.xls':'NG_GNB',
    'flight_Onshore Petroleum & Natural Gas Production.xls':'PROD_ON',
    'flight_Other Petroleum & Natural Gas Systems.xls':'ONG_OTH',
    'flight_Underground Natural Gas Storage.xls':'NG_STOR',
}

sub_sector_dict = {
    'NG_LDC':'NaturalGasDistribution',
    'PROD_OFF':'OffshoreProduction',
    'PROD_ON':'OnshoreProduction',
}

In [919]:
#pip install pyxlsb

In [920]:
#directory_path = 'C://Users//shashank.bekal//PythonWorkArea//Jnana-Research//OKC-Facilities//files'
directory_path = './/files'
all_files = os.listdir(directory_path)
xls_files = [file for file in all_files if file.endswith('.xls')]

print(xls_files)

['flight_Liquefied Natural Gas ImpExp Equipment.xls', 'flight_Liquefied Natural Gas Storage.xls', 'flight_Natural Gas Local Distribution Companies.xls', 'flight_Natural Gas Processing.xls', 'flight_Natural Gas TransmissionCompression.xls', 'flight_Offshore Petroleum & Natural Gas Production.xls', 'flight_Onshore Natural Gas Transmission Pipelines.xls', 'flight_Onshore Petroleum & Natural Gas Gathering & Boosting.xls', 'flight_Onshore Petroleum & Natural Gas Production.xls', 'flight_Other Petroleum & Natural Gas Systems.xls', 'flight_Underground Natural Gas Storage.xls']


In [921]:
facilities_master_df = pd.DataFrame()
for file_name in xls_files:
    file_dict = pd.read_excel(f'.//files/{file_name}',sheet_name=None,skiprows=6)
    file_df = pd.concat(file_dict, ignore_index=True)
    file_df["Abbr"] = abbr_dict[file_name]
    #print((file_name))
    facilities_master_df = pd.concat([facilities_master_df,file_df])
#print('---')
print(len(facilities_master_df))

C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\2884616571.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)
C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\2884616571.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)
C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\2884616571.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a 

30781


In [922]:
#df

In [923]:
required_subsectors = ['PROD_OFF','PROD_ON']

In [924]:
# filter by 3 required subsectors
facilities_master_df = facilities_master_df[facilities_master_df['Abbr'].isin(required_subsectors)]
facilities_master_df['sub_sector'] = facilities_master_df['Abbr'].map(sub_sector_dict)
facilities_master_df = facilities_master_df.rename(columns={'GHG QUANTITY (METRIC TONS CO2e)':'FACILITY_WIDE_GHG_QUANTITY'})
facilities_master_df = facilities_master_df.rename(columns={'GHGRP ID':'facility_id','REPORTING YEAR':'reporting_year'})
facilities_master_df = facilities_master_df.drop(['FACILITY NAME','SUBPARTS'],axis=1)

In [925]:
#generate_column_info(facilities_master_df)

In [926]:
#facilities_master_df

## Facilities Metadata

In [927]:
# facilities_metadata_df = pd.read_csv('rlps_ghg_emitter_facilities.csv')
# facilities_metadata_df = facilities_metadata_df[['facility_id','year','epa_verified']]
# facilities_metadata_df

In [928]:
# facilities_metadata_df.columns

In [929]:
# facilities_metadata_df[facilities_metadata_df.duplicated(subset=['facility_id','year'])]

In [930]:
# facilities_master_df = pd.merge(facilities_master_df,facilities_metadata_df,how='left',left_on=['GHGRP ID','REPORTING YEAR'],right_on=['facility_id','year'])

In [931]:
# facilities_master_df

In [932]:
# facilities_master_df.columns

In [933]:
# generate_column_info(facilities_master_df)

## Combustion Sources

In [934]:
#emissions_by_unit_df = pd.read_excel('emissions_by_unit_and_fuel_type_c_d_aa.xlsx',engine="openpyxl", sheet_name="UNIT_DATA")
c_df = pd.read_excel('emissions_by_unit_and_fuel_type_c_d_aa.xlsx', sheet_name="UNIT_DATA",skiprows=6)
c_df['Reporting Year'] = c_df['Reporting Year'].astype(int)
c_df['Facility Id'] = c_df['Facility Id'].astype(int)
c_df = c_df.rename(columns={'Facility Id':'facility_id','Facility Name':'facility_name','Reporting Year':'reporting_year'})
c_df = c_df.rename(columns={'Unit Type':'reporting_category','Unit CO2 emissions (non-biogenic) ':'total_co2_emissions','Unit Methane (CH4) emissions ':'total_ch4_emissions'})
c_df = c_df.rename(columns={'Unit Nitrous Oxide (N2O) emissions ':'total_n2o_emissions'})
c_df = c_df.drop(columns=['Unit Reporting Method','City','Primary NAICS Code','FRS Id','Industry Type (subparts)','Industry Type (sectors)','Unit Biogenic CO2 emissions (metric tons)'])
c_df = c_df.rename(columns={'Unit Maximum Rated Heat Input Capacity (mmBTU/hr)':'maximum_rated_heat_input_capacity','State':'state'})
#c_df = c_df[c_df['reporting_category'].notna()]
c_df['subpart'] = 'SubpartC'
c_df

,facility_id,facility_name,state,reporting_year,Unit Name,reporting_category,maximum_rated_heat_input_capacity,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,subpart
0,1001155,1500 South Tibbs LLC d/b/a Aurorium Indianapol...,IN,2023,GP-1,NaN,91.8,64740.5,30.50,36.356,SubpartC
1,1012147,17Z Gas Plant - Chevron USA Inc.,CA,2018,CP-03.00,OCS (Other combustion source),30.0,3304.7,1.50,1.788,SubpartC
2,1012147,17Z Gas Plant - Chevron USA Inc.,CA,2018,CP-03.01,OCS (Other combustion source),30.0,0.2,0.00,0.000,SubpartC
3,1012147,17Z Gas Plant - Chevron USA Inc.,CA,2018,CP-03.02,OCS (Other combustion source),30.0,175.0,0.00,0.000,SubpartC
4,1012147,17Z Gas Plant - Chevron USA Inc.,CA,2017,CP-03.00,OCS (Other combustion source),30.0,9106.1,4.25,5.066,SubpartC
...,...,...,...,...,...,...,...,...,...,...,...
268773,1003318,ZYBACH CRYOGENIC PLANT,TX,2015,GP-Zybach,OCS (Other combustion source),32.0,33757.8,16.00,19.072,SubpartC
268774,1003318,ZYBACH CRYOGENIC PLANT,TX,2014,GP-Zybach,OCS (Other combustion source),32.0,32901.2,15.50,18.476,SubpartC
268775,1003318,ZYBACH CRYOGENIC PLANT,TX,2013,GP-Zybach,OCS (Other combustion source),32.0,42700.7,20.25,24.138,SubpartC
268776,1003318,ZYBACH CRYOGENIC PLANT,TX,2012,GP-Zybach,OCS (Other combustion source),32.0,46840.7,22.00,26.224,SubpartC


# Emission Source

In [935]:
w_df = pd.read_csv('ef_w_emissions_source_ghg.csv',encoding = 'ISO-8859-1')
w_df['reporting_year'] = w_df['reporting_year'].astype(int)
w_df['facility_id'] = w_df['facility_id'].astype(int)
w_df = w_df.drop(columns=['bamm_desc_source_summary','bamm_indicator_source_summary','industry_segment'])
w_df = w_df.rename(columns={'total_reported_co2_emissions':'total_co2_emissions'})
w_df = w_df.rename(columns={'total_reported_ch4_emissions':'total_ch4_emissions'})
w_df = w_df.rename(columns={'total_reported_n2o_emissions':'total_n2o_emissions'})
w_df['total_ch4_emissions'] = w_df['total_ch4_emissions'] * 25
w_df['total_n2o_emissions'] = w_df['total_n2o_emissions'] * 298
w_df['subpart'] = 'SubpartW'
w_df

,basin_associated_with_facility,facility_id,facility_name,reporting_category,reporting_year,total_ch4_emissions,total_co2_emissions,total_n2o_emissions,subpart
0,NaN,1003447,UGC - DRAGON TRAIL GAS PLANT - 08-103-00036,Enhanced Oil Recovery Hydrocarbon Liquids [98....,2016,NaN,0.0,NaN,SubpartW
1,NaN,1004777,Great Lakes Gas Transmission Pipeline Shevlin ...,Completions and Workovers with Hydraulic Fract...,2023,0.00,0.0,0.0,SubpartW
2,"220 - Gulf Coast Basin (LA, TX)",1009706,"SilverBow Resources, LLC - Basin 220",Completions and Workovers without Hydraulic Fr...,2020,0.00,0.0,0.0,SubpartW
3,NaN,1012154,580 - San Juan Basin Gathering/Boosting,Dehydrators [98.236(e)],2018,94270.75,1223.8,0.0,SubpartW
4,NaN,1004849,CONROE NATURAL GAS PROCESSING PLANT,Reciprocating Compressors [98.236(p)],2015,1825.50,4.4,NaN,SubpartW
...,...,...,...,...,...,...,...,...,...
503055,NaN,1005688,STATION #7 - MOUNTAINAIR CMP STN,Natural Gas Driven Pneumatic Pumps [98.236(c)],2018,0.00,0.0,NaN,SubpartW
503056,NaN,1007441,Eunice Gas Plant,Combustion Equipment at Onshore Petroleum and ...,2021,0.00,0.0,0.0,SubpartW
503057,395 - Williston Basin,1010002,Murex Petroleum Corporation - Williston Basin 395,Completions and Workovers without Hydraulic Fr...,2019,0.00,0.0,0.0,SubpartW
503058,NaN,1012624,Repsol Pipelines LLC - Eagle Ford,Onshore Production [98.236(aa)(1)],2016,NaN,NaN,NaN,SubpartW


## ConCat c_df and w_df

In [936]:
df = pd.concat([c_df,w_df], ignore_index=True)

In [937]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,771838,0.000000,9564,int64
1,facility_name,0,771838,0.000000,10447,object
2,state,503060,268778,65.176889,54,object
3,reporting_year,0,771838,0.000000,14,int64
4,Unit Name,503060,268778,65.176889,21995,object
5,reporting_category,9103,762735,1.179393,117,object
6,maximum_rated_heat_input_capacity,556592,215246,72.112542,6996,float64
7,total_co2_emissions,46286,725552,5.996854,181252,float64
8,total_ch4_emissions,108182,663656,14.016154,55506,float64
9,total_n2o_emissions,250372,521466,32.438413,15814,float64


In [938]:
# df['total_ghg_emissions'] = df['total_co2_emissions'] + df['total_ch4_emissions'] + df['total_n2o_emissions']
# df = df[df['total_ghg_emissions']>0]

In [939]:
len(df)

771838

## Rename Reporting Category

In [940]:
reporting_category_names_df = pd.read_csv('reporting_categories.csv')
df = pd.merge(df, reporting_category_names_df, how='left',on=['reporting_category'])
df = df.drop('reporting_category',axis=1)
df = df.rename(columns={'REVISED_COLUMNNAME':'reporting_category'})

## Concat Pneumatic Devices from detailed dataset

In [941]:
len(df[df.reporting_category == 'NG_PneumaticDevices'])

24595

In [942]:
df.columns

Index(['facility_id', 'facility_name', 'state', 'reporting_year', 'Unit Name',
       'maximum_rated_heat_input_capacity', 'total_co2_emissions',
       'total_ch4_emissions', 'total_n2o_emissions', 'subpart',
       'basin_associated_with_facility', 'reporting_category'],
      dtype='object')

In [943]:
# Drop Pneumatic Devices from original dataset
df = df[df.reporting_category != 'NG_PneumaticDevices']

# add them from this csv file
pneumatic_devices_df = pd.read_csv('ef_w_ngpneumatic_dev_units.csv')
pneumatic_devices_df = pneumatic_devices_df[['facility_id','reporting_year','facility_name','estimated_number_of_hours','pneumatic_device_type','total_count','pneum_device_type_tot_co2_emm','total_ch4_emissions']]
pneumatic_devices_df=pneumatic_devices_df.rename(columns={'pneumatic_device_type':'Unit Name','estimated_number_of_hours':'pneu_est_no_of_hours','total_count':'pneu_dev_total_count','pneum_device_type_tot_co2_emm':'total_co2_emissions'})
pneumatic_devices_df['total_ch4_emissions'] = pneumatic_devices_df['total_ch4_emissions'] * 25
pneumatic_devices_df['total_n2o_emissions'] = 0
pneumatic_devices_df['reporting_category'] = 'NG_PneumaticDevices'
pneumatic_devices_df['subpart'] = 'SubpartW'

df = pd.concat([df,pneumatic_devices_df], ignore_index=True)
df = df.drop(columns=['state'])

## Concat AssociatedGasVentingFlaring from detailed dataset

In [944]:
len(df[df.reporting_category == 'AssociatedGasVentingFlaring'])

22328

In [945]:
# Drop AssociatedGasVentingFlaring from original dataset
df = df[df.reporting_category != 'AssociatedGasVentingFlaring']

In [946]:
# add them from this csv file
venting_flaring_df = pd.read_csv('ef_w_associated_ng_units.csv')

In [947]:
venting_flaring_df['total_co2_emissions'] =  venting_flaring_df['basin_ag_flaring_co2_emissions'] + venting_flaring_df['basin_ag_venting_co2_emissions']
venting_flaring_df['total_ch4_emissions'] =  venting_flaring_df['venting_ch4_emissions'] + venting_flaring_df['flaring_ch4_emissions']
venting_flaring_df = venting_flaring_df.drop(['basin_ag_flaring_co2_emissions','basin_ag_venting_co2_emissions','venting_ch4_emissions','flaring_ch4_emissions'],axis=1)
venting_flaring_df = venting_flaring_df.rename(columns={'flaring_n2o_emissions':'total_n2o_emissions'})
venting_flaring_df['total_ch4_emissions'] = venting_flaring_df['total_ch4_emissions'] * 25
venting_flaring_df['total_n2o_emissions'] = venting_flaring_df['total_n2o_emissions'] * 298
venting_flaring_df['reporting_category'] = 'AssociatedGasVentingFlaring'
venting_flaring_df['subpart'] = 'SubpartW'
venting_flaring_df = venting_flaring_df.drop(['facility_name','industry_segment','wells_two_year_delay'], axis=1)
df = pd.concat([df,venting_flaring_df], ignore_index=True)

## Concat AtmosphericTanks from detailed dataset

In [948]:
len(df[df.reporting_category == 'AtmosphericTanks'])

22328

In [949]:
# Drop AssociatedGasVentingFlaring from original dataset
df = df[df.reporting_category != 'AtmosphericTanks']

In [950]:
# add them from this csv file
athmospheric_tanks_df = pd.read_csv('ef_w_atm_stg_tanks_calc1or2.csv')

In [951]:
athmospheric_tanks_df['total_co2_emissions'] =  athmospheric_tanks_df['dump_valve_co2_emissions'] + athmospheric_tanks_df['flaring_co2_emissions'] + athmospheric_tanks_df['vapor_recovery_co2_emissions'] + athmospheric_tanks_df['venting_co2_emissions']
athmospheric_tanks_df['total_ch4_emissions'] =  athmospheric_tanks_df['dump_valve_ch4_emissions'] + athmospheric_tanks_df['flaring_ch4_emissions'] + athmospheric_tanks_df['vapor_recovery_ch4_emissions']+athmospheric_tanks_df['venting_ch4_emissions'] 
athmospheric_tanks_df = athmospheric_tanks_df.rename(columns={'flaring_n2o_emissions':'total_n2o_emissions'})
athmospheric_tanks_df = athmospheric_tanks_df.drop(['dump_valve_co2_emissions','flaring_co2_emissions','vapor_recovery_co2_emissions','venting_co2_emissions'],axis=1)
athmospheric_tanks_df = athmospheric_tanks_df.drop(['dump_valve_ch4_emissions','flaring_ch4_emissions','vapor_recovery_ch4_emissions','venting_ch4_emissions'],axis=1)
athmospheric_tanks_df['total_ch4_emissions'] = athmospheric_tanks_df['total_ch4_emissions'] * 25
athmospheric_tanks_df['total_n2o_emissions'] = athmospheric_tanks_df['total_n2o_emissions'] * 298
athmospheric_tanks_df=athmospheric_tanks_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','annual_ch4_recovered','annual_co2_recovered','atmospheric_tank_count',
                                             'average_a_p_i_gravity','average_separator_pressure','average_separator_temperature',
                                             'hrs_dump_valves_did_not_close','not_on_well_pad_tank_count','num_seps_with_stuck_dump_val',
                                             'number_of_wells_send_oil','tanks_venting_to_atm','tanks_with_flaring','tanks_with_vapor_recovery',
'total_volume_of_oil']]
athmospheric_tanks_df['reporting_category'] = 'AtmosphericTanks'
athmospheric_tanks_df['subpart'] = 'SubpartW'

In [952]:
athmospheric_tanks_2_df = pd.read_csv('ef_w_atm_stg_tanks_calc3.csv')
athmospheric_tanks_2_df['total_co2_emissions'] =  athmospheric_tanks_2_df['co2_emissions_from_flare'] + athmospheric_tanks_2_df['co2_emissions_not_flared']
athmospheric_tanks_2_df['total_ch4_emissions'] =  athmospheric_tanks_2_df['ch4_emissions_from_flare'] + athmospheric_tanks_2_df['ch4_emissions_not_flared']
athmospheric_tanks_2_df = athmospheric_tanks_2_df.drop(['co2_emissions_from_flare','co2_emissions_not_flared','ch4_emissions_from_flare','ch4_emissions_not_flared'],axis=1)
athmospheric_tanks_2_df = athmospheric_tanks_2_df.rename(columns={'n2o_emissions_from_flare':'total_n2o_emissions'})
athmospheric_tanks_2_df['total_ch4_emissions'] = athmospheric_tanks_2_df['total_ch4_emissions'] * 25
athmospheric_tanks_2_df['total_n2o_emissions'] = athmospheric_tanks_2_df['total_n2o_emissions'] * 298
athmospheric_tanks_2_df=athmospheric_tanks_2_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions',
                                                 'annual_oil_throughput','atmospheric_tank_count','fract_oil_throughput_flaring',
                                                'fract_throughput_with_vapor','tanks_with_flares','tanks_without_flares',
                                                'wells_with_seps','wells_without_seps']]
athmospheric_tanks_2_df['reporting_category'] = 'AtmosphericTanks'
athmospheric_tanks_2_df['subpart'] = 'SubpartW'

In [953]:
df = pd.concat([df,athmospheric_tanks_df,athmospheric_tanks_2_df], ignore_index=True)
len(df)

814638

In [954]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,814638,0.000000,9564,int64
1,facility_name,73111,741527,8.974661,10447,object
2,reporting_year,0,814638,0.000000,14,int64
3,Unit Name,506920,307718,62.226412,21998,object
4,maximum_rated_heat_input_capacity,599392,215246,73.577712,6996,float64
5,total_co2_emissions,136425,678213,16.746702,179707,float64
6,total_ch4_emissions,198334,616304,24.346274,52202,float64
7,total_n2o_emissions,271366,543272,33.311238,24956,float64
8,subpart,0,814638,0.000000,2,object
9,basin_associated_with_facility,680127,134511,83.488249,44,object


## Create total_ghg_emissions column

In [955]:
df['total_ghg_emissions'] = df['total_co2_emissions'] + df['total_ch4_emissions'] + df['total_n2o_emissions']
df = df[df['total_ghg_emissions']>0]

In [956]:
len(df)

326943

In [957]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,326943,0.000000,9467,int64
1,facility_name,1442,325501,0.441055,10342,object
2,reporting_year,0,326943,0.000000,14,int64
3,Unit Name,49818,277125,15.237518,21539,object
4,maximum_rated_heat_input_capacity,121593,205350,37.190886,6937,float64
5,total_co2_emissions,0,326943,0.000000,177279,float64
6,total_ch4_emissions,0,326943,0.000000,37677,float64
7,total_n2o_emissions,0,326943,0.000000,16369,float64
8,subpart,0,326943,0.000000,2,object
9,basin_associated_with_facility,295831,31112,90.483968,44,object


## Filter df to only required subsectors

In [958]:
facilities_in_required_subsectors = facilities_master_df['facility_id'].unique()

In [959]:
print(len(df))
df = df[df.facility_id.isin(facilities_in_required_subsectors)]
print(len(df))

326943
35784


## Join Facilities Metadata

In [960]:
df = pd.merge(df,facilities_master_df,how='left',on=['facility_id','reporting_year'])


In [961]:
#df

In [962]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,35784,0.000000,1148,object
1,facility_name,1442,34342,4.029734,1282,object
2,reporting_year,0,35784,0.000000,13,object
3,Unit Name,23118,12666,64.604292,459,object
4,maximum_rated_heat_input_capacity,29764,6020,83.176839,766,float64
5,total_co2_emissions,0,35784,0.000000,17578,float64
6,total_ch4_emissions,0,35784,0.000000,17155,float64
7,total_n2o_emissions,0,35784,0.000000,3235,float64
8,subpart,0,35784,0.000000,2,object
9,basin_associated_with_facility,15338,20446,42.862732,44,object


In [963]:
#df = df[df['GHG FACILITY_WIDE_GHG_QUANTITY']>0]

## Validate GHG Quantity for facility

In [966]:

validation_df = df
validation_df = validation_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
validation_df=validation_df.groupby(['facility_id','reporting_year']).sum().reset_index()
#validation_df

,facility_id,reporting_year,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,total_ghg_emissions
0,1000355,2011,3477.10,9793.9286,12.5929,13283.6215
1,1000355,2012,32826.46,35431.3094,38.4516,68296.2210
2,1000355,2013,57795.70,2597.0000,43.5000,60436.2000
3,1000355,2014,109502.80,2471.3000,88.0000,112062.1000
4,1000355,2015,157156.90,136306.0000,129.6300,293592.5300
...,...,...,...,...,...,...
8049,1014903,2023,29272.20,1138.0000,51.5540,30461.7540
8050,1014906,2023,43629.00,2539.7500,37.8460,46206.5960
8051,1014909,2023,336966.80,2437.7500,556.9620,339961.5120
8052,1014920,2023,14.00,5216.2500,0.0000,5230.2500


In [967]:
validation_df = pd.merge(validation_df,facilities_master_df[['facility_id','reporting_year','FACILITY_WIDE_GHG_QUANTITY']],how='left',on=['facility_id','reporting_year'])
#validation_df

,facility_id,reporting_year,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,total_ghg_emissions,FACILITY_WIDE_GHG_QUANTITY
0,1000355,2011,3477.10,9793.9286,12.5929,13283.6215,17728
1,1000355,2012,32826.46,35431.3094,38.4516,68296.2210,151519
2,1000355,2013,57795.70,2597.0000,43.5000,60436.2000,163014
3,1000355,2014,109502.80,2471.3000,88.0000,112062.1000,243117
4,1000355,2015,157156.90,136306.0000,129.6300,293592.5300,354851
...,...,...,...,...,...,...,...
8049,1014903,2023,29272.20,1138.0000,51.5540,30461.7540,30890
8050,1014906,2023,43629.00,2539.7500,37.8460,46206.5960,55011
8051,1014909,2023,336966.80,2437.7500,556.9620,339961.5120,415395
8052,1014920,2023,14.00,5216.2500,0.0000,5230.2500,25589


In [968]:
validation_df['diff'] = (validation_df['FACILITY_WIDE_GHG_QUANTITY']-validation_df['total_ghg_emissions']).abs()
validation_df['diff_percent_GHG']=100*validation_df['diff']/validation_df['FACILITY_WIDE_GHG_QUANTITY']
validation_df.to_csv('output_validation.csv', index=False)

,facility_id,reporting_year,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,total_ghg_emissions,FACILITY_WIDE_GHG_QUANTITY,diff,diff_percent_GHG
0,1000355,2011,3477.10,9793.9286,12.5929,13283.6215,17728,4444.3785,25.069825
1,1000355,2012,32826.46,35431.3094,38.4516,68296.2210,151519,83222.779,54.925639
2,1000355,2013,57795.70,2597.0000,43.5000,60436.2000,163014,102577.8,62.925761
3,1000355,2014,109502.80,2471.3000,88.0000,112062.1000,243117,131054.9,53.906103
4,1000355,2015,157156.90,136306.0000,129.6300,293592.5300,354851,61258.47,17.263153
...,...,...,...,...,...,...,...,...,...
8049,1014903,2023,29272.20,1138.0000,51.5540,30461.7540,30890,428.246,1.386358
8050,1014906,2023,43629.00,2539.7500,37.8460,46206.5960,55011,8804.404,16.004806
8051,1014909,2023,336966.80,2437.7500,556.9620,339961.5120,415395,75433.488,18.15946
8052,1014920,2023,14.00,5216.2500,0.0000,5230.2500,25589,20358.75,79.560553


In [863]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,35644,0.000000,1148,object
1,facility_name,1442,34202,4.045562,1280,object
2,reporting_year,0,35644,0.000000,13,object
3,Unit Name,23052,12592,64.672876,448,object
4,maximum_rated_heat_input_capacity,29685,5959,83.281899,760,float64
5,total_co2_emissions,0,35644,0.000000,17481,float64
6,total_ch4_emissions,0,35644,0.000000,17127,float64
7,total_n2o_emissions,0,35644,0.000000,3229,float64
8,subpart,0,35644,0.000000,2,object
9,basin_associated_with_facility,15211,20433,42.674784,44,object


In [864]:
df.to_csv('output.csv', index=False)

In [865]:
#AssociatedGasVentingFlaring

# AtmosphericTanks